<img src="GEOS_Logo.png" width="700" />

# Step 1 : <font color=blue>"download_files.ipynb"</font>
#### Aug 14, 2021  <font color=red>(still working)</font> 
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)



> list_full.dat (904 stations) and list_extra.dat (extra 3 stations) \
> total is 907 \
> At some point, a code that downloads (a)a station ID list and (b)a list of their coordinates will be ready \
> And then it will count how many stations, this GPS2FNL algorithm will process and pass this info to the later codes. \
> metadata is needed

> input is : list_default.dat


### **`GPS data processed by the NGL is available at:`**  [click here](http://geodesy.unr.edu)

> On the right side of the website, find <font color=red>"Downloadable Lists of GPS Data Holdings:"</font> \
> This algorithm uses data from "stations with 24 hour sample rate solutions, final orbits, 2 week latency." 



In [ ]:
# Ask if a user wants all stations (except MAGNET) or NOTA stations 
# If one choose to use all stations, the vertical position analysis must be done individually. 
# 

In [37]:
# Location Boundary
lon_min=235.43
lon_max=246.99
lat_min=31.31
lat_max=42.99

# Minimum number of daily position estimates
#if you want the NOTA stations, threshold_num = 0
#Other than that, threshold_num = 200
threshold_num=200

In [38]:
#import modules
import numpy as np
import pandas as pd
import urllib

1.  Read the station list
> url_lists="http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt"

2. Download metadate (discontinuous steps)
> url_metadata="http://geodesy.unr.edu/NGLStationPages/steps.txt"



In [13]:
# A list of GPS stations 
url_list="http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt" # URL
list_file="list_test.txt" # file name


# metadata for steps
url_metadata="http://geodesy.unr.edu/NGLStationPages/steps.txt" # URL
metadata_file="steps_test.txt" # file name

In [14]:
# Download the list of GPS stations (final orbits) and metadata 
urllib.request.urlretrieve(url_list, list_file)
urllib.request.urlretrieve(url_metadata, metadata_file)
print("Downloaded a GPS list and metadata for steps")

Downloaded a GPS list and metadata for steps


In [39]:
# Read the station list.
df_original_list=pd.read_csv(list_file ,header=None, sep=r'(?:,|\s+)', 
                comment='#', engine='python',
                names=['Sta','lat','lon','3','4','5','6','7','8','9','NumSol','11','12','13'])


# Here the first 4 columns will be used.
df_four_cols=df_original_list.loc[1:len(df_original_list),['Sta','lat','lon','NumSol']] #first three columns
df_four_cols=df_four_cols.reset_index(drop=True)

# str to float
df_four_cols['lon'] = df_four_cols['lon'].astype(float)
df_four_cols['lat'] = df_four_cols['lat'].astype(float)
df_four_cols['NumSol'] = df_four_cols['NumSol'].astype(int)

# Select stations within the region of interest.
# > lon_min,lon_max,lat_min,lat_max must be defined.

df_interest=df_four_cols.loc[(df_four_cols['lat'] >= lat_min)\
                            & (df_four_cols['lat'] <= lat_max)\
                            & (df_four_cols['lon'] >= lon_min)\
                            & (df_four_cols['lon'] <= lon_max)\
                            & (df_four_cols['NumSol'] >= threshold_num)]

df_interest=df_interest.reset_index(drop=True)

In [40]:
list_default="list_default.dat"
df_default=pd.read_csv(list_default, header=None, names=['Sta'])
df_default

,Sta
0,7ODM
1,ACSB
2,ACSX
3,AGMT
4,ALPP
...,...
901,WWFG
902,WWMT
903,YUHG
904,GOL2


In [41]:
#REMOVE MAGNET STATIONS FROM THE LIST

URL_MAGNET="http://geodesy.unr.edu/magnet/Table2web.html"
df_MAGNET=pd.read_html(URL_MAGNET, header=1, flavor = 'bs4')
df_MAGNET=df_MAGNET[0]
df_MAGNET.columns=['Sta','1','2','3','4','5','6']


df_no_MAGNAT = df_interest[~df_interest['Sta'].isin(df_MAGNET['Sta'])].reset_index(drop=True)
df_NOTA = df_interest[df_interest['Sta'].isin(df_default['Sta'])].reset_index(drop=True)

In [43]:
df_NOTA

,Sta,lat,lon,NumSol
0,7ODM,34.1164,242.9068,5931
1,ACSB,33.2743,242.5551,2791
2,ACSX,33.2743,242.5551,1920
3,AGMT,34.5943,243.5706,7848
4,ALPP,34.8245,241.3052,6328
...,...,...,...,...
900,WORG,35.6956,241.7576,2089
901,WRHS,33.9582,241.5724,7333
902,WWFG,33.2788,244.4212,1924
903,WWMT,33.9553,243.3461,6487
